<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_5_Reactive_Agent_Pattern_(LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Step 1: Install Required Packages

In [1]:
!pip install -q langgraph langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.7 MB/s eta 0:00:00


In [5]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


## Step 2: Import Modules

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "Reactive_Agent"
os.environ["LANGCHAIN_TRACING_V2"] = "true"


Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


##  Step 3: Define the LLM

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

<ipython-input-7-e7847f6b8c56>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


## Step 4: Define the State

In [8]:
from typing import TypedDict

class FAQState(TypedDict):
    question: str
    answer: str

## Step 5: Define the Node (Reactive Response

In [9]:
from langchain.prompts import ChatPromptTemplate

faq_prompt = ChatPromptTemplate.from_template(
    """You are a helpful support agent. Answer this customer FAQ:
Question: {question}

If it's unknown, respond with "Let me escalate that to our human team."
"""
)

faq_node = faq_prompt | llm


## Step 6: Define Transition Function

In [10]:
def update_answer(state: FAQState) -> FAQState:
    response = faq_node.invoke({"question": state["question"]})
    return {"question": state["question"], "answer": response.content}

##  Step 7: Build the Graph

In [11]:
builder = StateGraph(FAQState)
builder.add_node("faq_response", RunnableLambda(update_answer))
builder.set_entry_point("faq_response")
builder.set_finish_point("faq_response")

faq_graph = builder.compile()


## Step 8: Run the Graph

In [13]:
result = faq_graph.invoke({"question": "How long does shipping take?"})
print("Response:\n", result["answer"])

Response:
 Our standard shipping time is typically 3-5 business days. However, please keep in mind that shipping times may vary depending on your location and any potential delays with the carrier. If you have any concerns about the delivery timeframe, feel free to reach out to us for more specific information.
